## Documentation

To read more about the get API, visit the [docs](https://www.elastic.co/guide/en/elasticsearch/reference/current/docs-get.html).

![get_documents_docs](../images/get_documents_docs.png)

## Connect to ElasticSearch

In [2]:
from pprint import pprint
from elasticsearch import Elasticsearch

es = Elasticsearch('http://localhost:9200')
client_info = es.info()
print('Connected to Elasticsearch!')
pprint(client_info.body)

Connected to Elasticsearch!
{'cluster_name': 'es-docker-cluster',
 'cluster_uuid': '68vsKryIR7Ss49bLh7mz5Q',
 'name': 'es01',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2025-12-16T10:09:08.849001802Z',
             'build_flavor': 'default',
             'build_hash': 'd8972a71dbbd64ff17f2f4dba9ca2c3fe09fb100',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '10.3.2',
             'minimum_index_compatibility_version': '8.0.0',
             'minimum_wire_compatibility_version': '8.19.0',
             'number': '9.2.3'}}


## Index documents

In [3]:
es.indices.delete(index='my_index', ignore_unavailable=True)
es.indices.create(index='my_index')

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})

We are using the `dummy_data.json` file to insert multiple documents in the index. We store the id of each document in the `document_ids` list.

In [4]:
import json
from tqdm import tqdm


document_ids = []
dummy_data = json.load(open("../data/dummy_data.json"))
for document in tqdm(dummy_data, total=len(dummy_data)):
    response = es.index(index='my_index', body=document)
    document_ids.append(response['_id'])

100%|██████████| 3/3 [00:00<00:00, 31.37it/s]


In [5]:
document_ids

['Aj0ea5sBX61qEH3bTkkE', 'Az0ea5sBX61qEH3bTklW', 'BD0ea5sBX61qEH3bTkld']

## Get operation

This is an example of a successful operation. If the `ID` exists in the document, the `get` operation won't return any errors.

In [6]:
response = es.get(index='my_index', id=document_ids[0])

In [ ]:
#!/bin/sh
curl -X GET "http://localhost:9200/my_index/_doc/_z0Ya5sBX61qEH3bHUio"   -- .../_id

# Kibana Dev Tools equivalent:
GET /my_index/_doc/_z0Ya5sBX61qEH3bHUio   -- GET /my_index/_doc/_id

In [7]:
from pprint import pprint

pprint(response.body)

{'_id': 'Aj0ea5sBX61qEH3bTkkE',
 '_index': 'my_index',
 '_primary_term': 1,
 '_seq_no': 0,
 '_source': {'created_on': '2024-09-22',
             'text': 'This is the first sample document text.',
             'title': 'Sample Title 1'},
 '_version': 1,
 'found': True}


This example shows that the `get` operation fails when providing it with an `ID` that does not exist in the index.

In [8]:
try:
    response = es.get(index='my_index', id="id")
except Exception as e:
    print(e)

NotFoundError(404, "{'_index': 'my_index', '_id': 'id', 'found': False}")
